In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from gensim.models import Word2Vec, Phrases
from gensim.models.phrases import Phraser

C:\Users\gohw\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
stop = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gohw\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gohw\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, Flatten
from keras.initializers import Constant
from keras.optimizers import Adam

Using TensorFlow backend.


In [4]:
df = pd.read_csv('train.csv')

In [5]:
ids_with_target_error = [328,443,513,2619,3640,3900,4342,5781,6552,6554,6570,6701,6702,6729,6861,7226]

In [6]:
df.loc[df['id'].isin(ids_with_target_error),'target'] = 0

# Count Vectorizer

In [7]:
countvec = feature_extraction.text.CountVectorizer()
tfidfvec = feature_extraction.text.TfidfVectorizer()

In [8]:
model = linear_model.RidgeClassifier()

In [9]:
vector_count = countvec.fit_transform(df['text']).todense()
vector_tfidf = tfidfvec.fit_transform(df['text']).todense()

In [11]:
scores = model_selection.cross_val_score(model, vector_count, df['target'], cv=5, scoring='f1')
np.mean(scores)

0.5715569538004827

In [12]:
scores = model_selection.cross_val_score(model, vector_tfidf, df['target'], cv=5, scoring='f1')
np.mean(scores)

0.6325075863982154

# Glove

In [7]:
def create_corpus(df):
    corpus = []
    for row in tqdm(df['text']):
        word = [w.lower() for w in word_tokenize(row) if ((w.isalpha()) and (w not in stop)) ]
        corpus.append(word)
    return corpus

In [8]:
corpus = create_corpus(df)

100%|████████████████████████████████████| 7613/7613 [00:01<00:00, 4785.94it/s]


In [56]:
embedding_dict = {}
with open('glove.6B/glove.6B.300d.txt','r', encoding='utf8') as f:
    for line in f:
        val = line.split()
        word = val[0]
        vec = np.array(val[1:],'float32')
        embedding_dict[word] = vec
f.close()

In [57]:
tok = Tokenizer()
tok.fit_on_texts(corpus)
seq = tok.texts_to_sequences(corpus)

In [58]:
pad = pad_sequences(seq, maxlen=30, truncating='post', padding='post')

In [59]:
word_index = tok.word_index
len(word_index)

15015

In [60]:
emb_mat = np.zeros((len(word_index)+1,300))
for word,i in word_index.items():
    emb_vec = embedding_dict.get(word)
    if emb_vec is not None:
        emb_mat[i] = emb_vec

In [61]:
emb_mat.shape

(15016, 300)

In [62]:
X_train, X_test, y_train, y_test = train_test_split(pad,df['target'],test_size=0.2, shuffle=True)

In [63]:
model = Sequential()
embedding = Embedding(emb_mat.shape[0], emb_mat.shape[1], embeddings_initializer=Constant(emb_mat), input_length=30, trainable=False)
model.add(embedding)
#model.add(SpatialDropout1D(0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
#model.add(Flatten())
#model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
opt = Adam(lr=1e-5)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [64]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 30, 300)           4504800   
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                93440     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 4,598,305
Trainable params: 93,505
Non-trainable params: 4,504,800
_________________________________________________________________


In [65]:
history = model.fit(X_train, y_train, batch_size=4, epochs=10, validation_data=(X_test, y_test), verbose=1)

Train on 6090 samples, validate on 1523 samples
Epoch 1/10


1568/6090 [======>.......................] - ETA: 1:26:36 - loss: 0.6934 - acc: 0.25 - ETA: 43:58 - loss: 0.6944 - acc: 0.2500 - ETA: 22:18 - loss: 0.6935 - acc: 0.43 - ETA: 15:09 - loss: 0.6935 - acc: 0.50 - ETA: 11:32 - loss: 0.6934 - acc: 0.53 - ETA: 9:22 - loss: 0.6934 - acc: 0.5250 - ETA: 7:55 - loss: 0.6933 - acc: 0.520 - ETA: 6:53 - loss: 0.6932 - acc: 0.535 - ETA: 6:07 - loss: 0.6931 - acc: 0.531 - ETA: 5:31 - loss: 0.6929 - acc: 0.569 - ETA: 5:02 - loss: 0.6929 - acc: 0.587 - ETA: 4:38 - loss: 0.6930 - acc: 0.556 - ETA: 4:19 - loss: 0.6929 - acc: 0.541 - ETA: 4:02 - loss: 0.6928 - acc: 0.538 - ETA: 3:48 - loss: 0.6929 - acc: 0.526 - ETA: 3:36 - loss: 0.6929 - acc: 0.525 - ETA: 3:25 - loss: 0.6928 - acc: 0.531 - ETA: 3:15 - loss: 0.6929 - acc: 0.514 - ETA: 3:07 - loss: 0.6930 - acc: 0.506 - ETA: 2:59 - loss: 0.6930 - acc: 0.526 - ETA: 2:52 - loss: 0.6930 - acc: 0.518 - ETA: 2:45 - loss: 0.6930 - acc: 0.517 - ETA: 2:40 - loss: 0.6931 - acc: 0.511 - ETA: 2:35 - loss: 0.6931 - acc

3184/6090 [==============>...............] - ETA: 47s - loss: 0.6930 - acc: 0.54 - ETA: 47s - loss: 0.6930 - acc: 0.54 - ETA: 47s - loss: 0.6930 - acc: 0.54 - ETA: 47s - loss: 0.6930 - acc: 0.54 - ETA: 46s - loss: 0.6930 - acc: 0.54 - ETA: 46s - loss: 0.6930 - acc: 0.54 - ETA: 46s - loss: 0.6930 - acc: 0.54 - ETA: 46s - loss: 0.6930 - acc: 0.54 - ETA: 46s - loss: 0.6930 - acc: 0.54 - ETA: 46s - loss: 0.6930 - acc: 0.54 - ETA: 46s - loss: 0.6930 - acc: 0.54 - ETA: 46s - loss: 0.6930 - acc: 0.54 - ETA: 46s - loss: 0.6930 - acc: 0.54 - ETA: 46s - loss: 0.6929 - acc: 0.54 - ETA: 46s - loss: 0.6929 - acc: 0.54 - ETA: 45s - loss: 0.6929 - acc: 0.54 - ETA: 45s - loss: 0.6929 - acc: 0.54 - ETA: 45s - loss: 0.6930 - acc: 0.54 - ETA: 45s - loss: 0.6930 - acc: 0.54 - ETA: 45s - loss: 0.6929 - acc: 0.54 - ETA: 45s - loss: 0.6930 - acc: 0.54 - ETA: 45s - loss: 0.6930 - acc: 0.54 - ETA: 45s - loss: 0.6930 - acc: 0.54 - ETA: 45s - loss: 0.6930 - acc: 0.54 - ETA: 44s - loss: 0.6930 - acc: 0.54 - ETA: 

4864/6090 [======================>.......] - ETA: 27s - loss: 0.6926 - acc: 0.56 - ETA: 27s - loss: 0.6926 - acc: 0.56 - ETA: 27s - loss: 0.6926 - acc: 0.56 - ETA: 27s - loss: 0.6926 - acc: 0.56 - ETA: 27s - loss: 0.6926 - acc: 0.56 - ETA: 27s - loss: 0.6926 - acc: 0.56 - ETA: 27s - loss: 0.6926 - acc: 0.56 - ETA: 27s - loss: 0.6926 - acc: 0.56 - ETA: 27s - loss: 0.6926 - acc: 0.56 - ETA: 27s - loss: 0.6926 - acc: 0.56 - ETA: 27s - loss: 0.6926 - acc: 0.56 - ETA: 27s - loss: 0.6926 - acc: 0.56 - ETA: 27s - loss: 0.6926 - acc: 0.56 - ETA: 27s - loss: 0.6926 - acc: 0.56 - ETA: 26s - loss: 0.6926 - acc: 0.56 - ETA: 26s - loss: 0.6926 - acc: 0.56 - ETA: 26s - loss: 0.6926 - acc: 0.56 - ETA: 26s - loss: 0.6926 - acc: 0.56 - ETA: 26s - loss: 0.6925 - acc: 0.56 - ETA: 26s - loss: 0.6926 - acc: 0.56 - ETA: 26s - loss: 0.6925 - acc: 0.56 - ETA: 26s - loss: 0.6925 - acc: 0.56 - ETA: 26s - loss: 0.6925 - acc: 0.56 - ETA: 26s - loss: 0.6925 - acc: 0.56 - ETA: 26s - loss: 0.6925 - acc: 0.56 - ETA: 

6090/6090 [==============================] - ETA: 11s - loss: 0.6920 - acc: 0.56 - ETA: 11s - loss: 0.6921 - acc: 0.56 - ETA: 11s - loss: 0.6921 - acc: 0.56 - ETA: 10s - loss: 0.6921 - acc: 0.56 - ETA: 10s - loss: 0.6921 - acc: 0.56 - ETA: 10s - loss: 0.6921 - acc: 0.56 - ETA: 10s - loss: 0.6920 - acc: 0.56 - ETA: 10s - loss: 0.6920 - acc: 0.56 - ETA: 10s - loss: 0.6920 - acc: 0.56 - ETA: 10s - loss: 0.6920 - acc: 0.56 - ETA: 10s - loss: 0.6920 - acc: 0.56 - ETA: 10s - loss: 0.6920 - acc: 0.56 - ETA: 9s - loss: 0.6920 - acc: 0.5675 - ETA: 9s - loss: 0.6921 - acc: 0.567 - ETA: 9s - loss: 0.6921 - acc: 0.566 - ETA: 9s - loss: 0.6921 - acc: 0.567 - ETA: 9s - loss: 0.6921 - acc: 0.566 - ETA: 9s - loss: 0.6921 - acc: 0.567 - ETA: 9s - loss: 0.6920 - acc: 0.567 - ETA: 9s - loss: 0.6920 - acc: 0.567 - ETA: 9s - loss: 0.6920 - acc: 0.567 - ETA: 9s - loss: 0.6920 - acc: 0.567 - ETA: 9s - loss: 0.6920 - acc: 0.567 - ETA: 8s - loss: 0.6920 - acc: 0.567 - ETA: 8s - loss: 0.6920 - acc: 0.567 - ETA:

...

1328/6090 [=====>........................] - ETA: 47s - loss: 0.1813 - acc: 1.00 - ETA: 55s - loss: 0.3658 - acc: 0.91 - ETA: 56s - loss: 0.3138 - acc: 0.90 - ETA: 57s - loss: 0.5104 - acc: 0.75 - ETA: 58s - loss: 0.4872 - acc: 0.77 - ETA: 58s - loss: 0.4795 - acc: 0.77 - ETA: 57s - loss: 0.4565 - acc: 0.80 - ETA: 59s - loss: 0.4609 - acc: 0.80 - ETA: 58s - loss: 0.4513 - acc: 0.81 - ETA: 58s - loss: 0.4501 - acc: 0.81 - ETA: 57s - loss: 0.4573 - acc: 0.80 - ETA: 57s - loss: 0.4483 - acc: 0.80 - ETA: 56s - loss: 0.4537 - acc: 0.79 - ETA: 57s - loss: 0.4461 - acc: 0.79 - ETA: 57s - loss: 0.4628 - acc: 0.79 - ETA: 58s - loss: 0.4562 - acc: 0.80 - ETA: 58s - loss: 0.4617 - acc: 0.79 - ETA: 59s - loss: 0.4653 - acc: 0.79 - ETA: 59s - loss: 0.4561 - acc: 0.80 - ETA: 1:00 - loss: 0.4591 - acc: 0.801 - ETA: 1:00 - loss: 0.4590 - acc: 0.805 - ETA: 1:00 - loss: 0.4783 - acc: 0.796 - ETA: 1:00 - loss: 0.5105 - acc: 0.775 - ETA: 1:00 - loss: 0.5109 - acc: 0.774 - ETA: 1:00 - loss: 0.5100 - acc: 0

2684/6090 [============>.................] - ETA: 54s - loss: 0.4847 - acc: 0.79 - ETA: 54s - loss: 0.4847 - acc: 0.79 - ETA: 54s - loss: 0.4831 - acc: 0.79 - ETA: 54s - loss: 0.4825 - acc: 0.79 - ETA: 54s - loss: 0.4817 - acc: 0.79 - ETA: 54s - loss: 0.4810 - acc: 0.79 - ETA: 54s - loss: 0.4816 - acc: 0.79 - ETA: 54s - loss: 0.4812 - acc: 0.79 - ETA: 54s - loss: 0.4823 - acc: 0.79 - ETA: 54s - loss: 0.4843 - acc: 0.79 - ETA: 54s - loss: 0.4858 - acc: 0.79 - ETA: 54s - loss: 0.4878 - acc: 0.79 - ETA: 54s - loss: 0.4871 - acc: 0.79 - ETA: 54s - loss: 0.4869 - acc: 0.79 - ETA: 54s - loss: 0.4870 - acc: 0.79 - ETA: 54s - loss: 0.4868 - acc: 0.79 - ETA: 54s - loss: 0.4861 - acc: 0.79 - ETA: 54s - loss: 0.4864 - acc: 0.79 - ETA: 54s - loss: 0.4859 - acc: 0.79 - ETA: 54s - loss: 0.4877 - acc: 0.79 - ETA: 54s - loss: 0.4885 - acc: 0.78 - ETA: 54s - loss: 0.4900 - acc: 0.78 - ETA: 54s - loss: 0.4901 - acc: 0.78 - ETA: 54s - loss: 0.4904 - acc: 0.78 - ETA: 54s - loss: 0.4906 - acc: 0.78 - ETA: 

4028/6090 [==================>...........] - ETA: 39s - loss: 0.4770 - acc: 0.79 - ETA: 38s - loss: 0.4766 - acc: 0.79 - ETA: 38s - loss: 0.4777 - acc: 0.79 - ETA: 38s - loss: 0.4773 - acc: 0.79 - ETA: 38s - loss: 0.4772 - acc: 0.79 - ETA: 38s - loss: 0.4772 - acc: 0.79 - ETA: 38s - loss: 0.4767 - acc: 0.79 - ETA: 38s - loss: 0.4763 - acc: 0.79 - ETA: 38s - loss: 0.4756 - acc: 0.79 - ETA: 38s - loss: 0.4757 - acc: 0.79 - ETA: 38s - loss: 0.4757 - acc: 0.79 - ETA: 38s - loss: 0.4757 - acc: 0.79 - ETA: 37s - loss: 0.4762 - acc: 0.79 - ETA: 37s - loss: 0.4765 - acc: 0.79 - ETA: 37s - loss: 0.4761 - acc: 0.79 - ETA: 37s - loss: 0.4757 - acc: 0.79 - ETA: 37s - loss: 0.4756 - acc: 0.79 - ETA: 37s - loss: 0.4753 - acc: 0.79 - ETA: 37s - loss: 0.4759 - acc: 0.79 - ETA: 37s - loss: 0.4762 - acc: 0.79 - ETA: 37s - loss: 0.4758 - acc: 0.79 - ETA: 37s - loss: 0.4754 - acc: 0.79 - ETA: 37s - loss: 0.4753 - acc: 0.79 - ETA: 37s - loss: 0.4757 - acc: 0.79 - ETA: 37s - loss: 0.4758 - acc: 0.79 - ETA: 

5312/6090 [=========================>....] - ETA: 24s - loss: 0.4777 - acc: 0.79 - ETA: 24s - loss: 0.4781 - acc: 0.79 - ETA: 23s - loss: 0.4779 - acc: 0.79 - ETA: 23s - loss: 0.4784 - acc: 0.79 - ETA: 23s - loss: 0.4784 - acc: 0.79 - ETA: 23s - loss: 0.4787 - acc: 0.79 - ETA: 23s - loss: 0.4788 - acc: 0.79 - ETA: 23s - loss: 0.4790 - acc: 0.79 - ETA: 23s - loss: 0.4790 - acc: 0.79 - ETA: 23s - loss: 0.4792 - acc: 0.79 - ETA: 23s - loss: 0.4795 - acc: 0.79 - ETA: 23s - loss: 0.4799 - acc: 0.79 - ETA: 23s - loss: 0.4800 - acc: 0.79 - ETA: 23s - loss: 0.4797 - acc: 0.79 - ETA: 23s - loss: 0.4797 - acc: 0.79 - ETA: 22s - loss: 0.4797 - acc: 0.79 - ETA: 22s - loss: 0.4796 - acc: 0.79 - ETA: 22s - loss: 0.4793 - acc: 0.79 - ETA: 22s - loss: 0.4795 - acc: 0.79 - ETA: 22s - loss: 0.4796 - acc: 0.79 - ETA: 22s - loss: 0.4793 - acc: 0.79 - ETA: 22s - loss: 0.4794 - acc: 0.79 - ETA: 22s - loss: 0.4797 - acc: 0.79 - ETA: 22s - loss: 0.4798 - acc: 0.79 - ETA: 22s - loss: 0.4809 - acc: 0.79 - ETA: 

6090/6090 [==============================] - ETA: 9s - loss: 0.4814 - acc: 0.793 - ETA: 9s - loss: 0.4811 - acc: 0.793 - ETA: 9s - loss: 0.4811 - acc: 0.793 - ETA: 9s - loss: 0.4810 - acc: 0.793 - ETA: 8s - loss: 0.4812 - acc: 0.793 - ETA: 8s - loss: 0.4816 - acc: 0.793 - ETA: 8s - loss: 0.4814 - acc: 0.793 - ETA: 8s - loss: 0.4816 - acc: 0.793 - ETA: 8s - loss: 0.4814 - acc: 0.793 - ETA: 8s - loss: 0.4813 - acc: 0.793 - ETA: 8s - loss: 0.4813 - acc: 0.793 - ETA: 8s - loss: 0.4813 - acc: 0.793 - ETA: 8s - loss: 0.4818 - acc: 0.793 - ETA: 8s - loss: 0.4822 - acc: 0.793 - ETA: 8s - loss: 0.4824 - acc: 0.793 - ETA: 7s - loss: 0.4827 - acc: 0.793 - ETA: 7s - loss: 0.4827 - acc: 0.793 - ETA: 7s - loss: 0.4830 - acc: 0.793 - ETA: 7s - loss: 0.4831 - acc: 0.793 - ETA: 7s - loss: 0.4830 - acc: 0.792 - ETA: 7s - loss: 0.4826 - acc: 0.793 - ETA: 7s - loss: 0.4823 - acc: 0.793 - ETA: 7s - loss: 0.4821 - acc: 0.793 - ETA: 7s - loss: 0.4822 - acc: 0.793 - ETA: 7s - loss: 0.4821 - acc: 0.793 - ETA: 

In [66]:
pred=model.predict(X_test)

In [67]:
f1_score(y_test.as_matrix(),np.round(pred).astype(int))

C:\Users\gohw\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


0.7441860465116279

# Word2Vec embedding

In [68]:
model = Word2Vec(corpus, iter=100, min_count=1, size=300, window=10)

In [69]:
len(model.wv.index2word)

15015

In [70]:
model.wv.syn0.shape

C:\Users\gohw\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


(15015, 300)

In [71]:
tok = Tokenizer()
tok.fit_on_texts(corpus)
seq = tok.texts_to_sequences(corpus)
pad = pad_sequences(seq, maxlen=30, truncating='post', padding='post')

In [72]:
word_index = tok.word_index
len(word_index)

15015

In [73]:
len(model.wv.syn0[2])

C:\Users\gohw\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


300

In [74]:
emb_mat_new = np.zeros((len(word_index)+1,300))
for word,i in word_index.items():
    index = model.wv.index2word.index(word)
    emb_vec_new = model.wv.syn0[index]
    if emb_vec_new is not None:
        emb_mat_new[i] = emb_vec_new

C:\Users\gohw\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  after removing the cwd from sys.path.


In [75]:
print(emb_mat_new.shape)
emb_mat_new

(15016, 300)


array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-2.33808666e-01, -1.28938818e+00,  5.79067171e-01, ...,
         4.45988141e-02,  1.43443614e-01, -2.23751500e-01],
       [-8.14709008e-01,  1.03409655e-01, -1.18410933e+00, ...,
         1.26772749e+00,  1.39811024e-01,  8.63084853e-01],
       ...,
       [-1.44951671e-01,  3.16515118e-02, -1.54727206e-01, ...,
         3.08250207e-02,  1.79997817e-01, -1.25375405e-01],
       [-6.60575926e-04,  1.19019702e-01, -1.54966917e-02, ...,
         5.82552701e-02,  8.06757137e-02,  4.43036892e-02],
       [ 1.00453962e-02,  9.98999402e-02, -1.16862990e-02, ...,
         4.10295613e-02,  6.75621107e-02,  5.01458235e-02]])

In [76]:
model = Sequential()
embedding = Embedding(emb_mat_new.shape[0], emb_mat_new.shape[1], embeddings_initializer=Constant(emb_mat_new), input_length=30, trainable=False)
model.add(embedding)
#model.add(SpatialDropout1D(0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
#model.add(Flatten())
#model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
opt = Adam(lr=1e-5)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [77]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 30, 300)           4504800   
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                93440     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 4,598,305
Trainable params: 93,505
Non-trainable params: 4,504,800
_________________________________________________________________


In [78]:
history = model.fit(X_train, y_train, batch_size=4, epochs=10, validation_data=(X_test, y_test))

Train on 6090 samples, validate on 1523 samples
Epoch 1/10


1352/6090 [=====>........................] - ETA: 4:03:13 - loss: 0.6922 - acc: 0.50 - ETA: 2:03:20 - loss: 0.6934 - acc: 0.37 - ETA: 1:02:04 - loss: 0.6930 - acc: 0.37 - ETA: 41:39 - loss: 0.6928 - acc: 0.5000 - ETA: 35:57 - loss: 0.6930 - acc: 0.50 - ETA: 28:09 - loss: 0.6934 - acc: 0.47 - ETA: 23:11 - loss: 0.6936 - acc: 0.45 - ETA: 19:45 - loss: 0.6933 - acc: 0.50 - ETA: 17:14 - loss: 0.6932 - acc: 0.50 - ETA: 15:18 - loss: 0.6928 - acc: 0.51 - ETA: 13:48 - loss: 0.6930 - acc: 0.53 - ETA: 12:33 - loss: 0.6930 - acc: 0.54 - ETA: 11:32 - loss: 0.6930 - acc: 0.54 - ETA: 10:40 - loss: 0.6931 - acc: 0.54 - ETA: 9:57 - loss: 0.6930 - acc: 0.5370 - ETA: 9:19 - loss: 0.6930 - acc: 0.525 - ETA: 8:47 - loss: 0.6929 - acc: 0.516 - ETA: 8:18 - loss: 0.6930 - acc: 0.515 - ETA: 7:52 - loss: 0.6930 - acc: 0.507 - ETA: 7:29 - loss: 0.6930 - acc: 0.500 - ETA: 7:09 - loss: 0.6930 - acc: 0.493 - ETA: 7:00 - loss: 0.6930 - acc: 0.500 - ETA: 6:42 - loss: 0.6930 - acc: 0.511 - ETA: 6:26 - loss: 0.6930 -

2784/6090 [============>.................] - ETA: 1:26 - loss: 0.6926 - acc: 0.562 - ETA: 1:26 - loss: 0.6926 - acc: 0.562 - ETA: 1:26 - loss: 0.6926 - acc: 0.562 - ETA: 1:26 - loss: 0.6926 - acc: 0.561 - ETA: 1:25 - loss: 0.6926 - acc: 0.560 - ETA: 1:25 - loss: 0.6927 - acc: 0.560 - ETA: 1:25 - loss: 0.6926 - acc: 0.560 - ETA: 1:24 - loss: 0.6927 - acc: 0.561 - ETA: 1:24 - loss: 0.6926 - acc: 0.562 - ETA: 1:24 - loss: 0.6926 - acc: 0.563 - ETA: 1:24 - loss: 0.6926 - acc: 0.563 - ETA: 1:23 - loss: 0.6926 - acc: 0.562 - ETA: 1:23 - loss: 0.6926 - acc: 0.562 - ETA: 1:22 - loss: 0.6926 - acc: 0.562 - ETA: 1:22 - loss: 0.6926 - acc: 0.562 - ETA: 1:22 - loss: 0.6926 - acc: 0.561 - ETA: 1:22 - loss: 0.6926 - acc: 0.563 - ETA: 1:21 - loss: 0.6926 - acc: 0.563 - ETA: 1:21 - loss: 0.6926 - acc: 0.562 - ETA: 1:21 - loss: 0.6926 - acc: 0.563 - ETA: 1:20 - loss: 0.6926 - acc: 0.563 - ETA: 1:20 - loss: 0.6926 - acc: 0.563 - ETA: 1:20 - loss: 0.6926 - acc: 0.563 - ETA: 1:20 - loss: 0.6926 - acc: 0.5

4156/6090 [===================>..........] - ETA: 47s - loss: 0.6921 - acc: 0.57 - ETA: 47s - loss: 0.6921 - acc: 0.57 - ETA: 47s - loss: 0.6921 - acc: 0.57 - ETA: 47s - loss: 0.6921 - acc: 0.57 - ETA: 47s - loss: 0.6921 - acc: 0.57 - ETA: 47s - loss: 0.6921 - acc: 0.57 - ETA: 47s - loss: 0.6921 - acc: 0.57 - ETA: 47s - loss: 0.6921 - acc: 0.57 - ETA: 46s - loss: 0.6921 - acc: 0.57 - ETA: 46s - loss: 0.6921 - acc: 0.57 - ETA: 46s - loss: 0.6921 - acc: 0.57 - ETA: 46s - loss: 0.6921 - acc: 0.57 - ETA: 46s - loss: 0.6921 - acc: 0.57 - ETA: 46s - loss: 0.6921 - acc: 0.57 - ETA: 46s - loss: 0.6921 - acc: 0.57 - ETA: 46s - loss: 0.6921 - acc: 0.57 - ETA: 46s - loss: 0.6921 - acc: 0.57 - ETA: 46s - loss: 0.6921 - acc: 0.57 - ETA: 46s - loss: 0.6921 - acc: 0.57 - ETA: 45s - loss: 0.6921 - acc: 0.57 - ETA: 45s - loss: 0.6921 - acc: 0.57 - ETA: 45s - loss: 0.6921 - acc: 0.57 - ETA: 45s - loss: 0.6921 - acc: 0.57 - ETA: 45s - loss: 0.6921 - acc: 0.57 - ETA: 45s - loss: 0.6921 - acc: 0.57 - ETA: 

5560/6090 [==========================>...] - ETA: 26s - loss: 0.6918 - acc: 0.57 - ETA: 26s - loss: 0.6917 - acc: 0.57 - ETA: 26s - loss: 0.6918 - acc: 0.57 - ETA: 25s - loss: 0.6918 - acc: 0.57 - ETA: 25s - loss: 0.6918 - acc: 0.57 - ETA: 25s - loss: 0.6918 - acc: 0.57 - ETA: 25s - loss: 0.6918 - acc: 0.57 - ETA: 25s - loss: 0.6918 - acc: 0.57 - ETA: 25s - loss: 0.6918 - acc: 0.57 - ETA: 25s - loss: 0.6918 - acc: 0.57 - ETA: 25s - loss: 0.6918 - acc: 0.57 - ETA: 25s - loss: 0.6918 - acc: 0.57 - ETA: 25s - loss: 0.6918 - acc: 0.57 - ETA: 24s - loss: 0.6918 - acc: 0.57 - ETA: 24s - loss: 0.6917 - acc: 0.57 - ETA: 24s - loss: 0.6918 - acc: 0.57 - ETA: 24s - loss: 0.6917 - acc: 0.57 - ETA: 24s - loss: 0.6917 - acc: 0.57 - ETA: 24s - loss: 0.6917 - acc: 0.57 - ETA: 24s - loss: 0.6917 - acc: 0.57 - ETA: 24s - loss: 0.6917 - acc: 0.57 - ETA: 23s - loss: 0.6917 - acc: 0.57 - ETA: 23s - loss: 0.6917 - acc: 0.57 - ETA: 23s - loss: 0.6917 - acc: 0.57 - ETA: 23s - loss: 0.6917 - acc: 0.57 - ETA: 

...

3308/6090 [===============>..............] - ETA: 1:03 - loss: 0.4985 - acc: 0.776 - ETA: 1:03 - loss: 0.4989 - acc: 0.775 - ETA: 1:03 - loss: 0.4988 - acc: 0.775 - ETA: 1:03 - loss: 0.4984 - acc: 0.776 - ETA: 1:03 - loss: 0.4987 - acc: 0.776 - ETA: 1:03 - loss: 0.4992 - acc: 0.776 - ETA: 1:03 - loss: 0.5003 - acc: 0.775 - ETA: 1:03 - loss: 0.5006 - acc: 0.775 - ETA: 1:03 - loss: 0.5015 - acc: 0.774 - ETA: 1:03 - loss: 0.5022 - acc: 0.774 - ETA: 1:03 - loss: 0.5025 - acc: 0.774 - ETA: 1:03 - loss: 0.5036 - acc: 0.773 - ETA: 1:03 - loss: 0.5042 - acc: 0.773 - ETA: 1:03 - loss: 0.5051 - acc: 0.772 - ETA: 1:02 - loss: 0.5052 - acc: 0.772 - ETA: 1:02 - loss: 0.5052 - acc: 0.772 - ETA: 1:02 - loss: 0.5050 - acc: 0.772 - ETA: 1:02 - loss: 0.5050 - acc: 0.772 - ETA: 1:02 - loss: 0.5051 - acc: 0.772 - ETA: 1:02 - loss: 0.5048 - acc: 0.772 - ETA: 1:02 - loss: 0.5046 - acc: 0.772 - ETA: 1:02 - loss: 0.5044 - acc: 0.772 - ETA: 1:02 - loss: 0.5041 - acc: 0.772 - ETA: 1:02 - loss: 0.5041 - acc: 0.7

4172/6090 [===================>..........] - ETA: 49s - loss: 0.5012 - acc: 0.77 - ETA: 49s - loss: 0.5010 - acc: 0.77 - ETA: 49s - loss: 0.5010 - acc: 0.77 - ETA: 48s - loss: 0.5010 - acc: 0.77 - ETA: 48s - loss: 0.5009 - acc: 0.77 - ETA: 48s - loss: 0.5011 - acc: 0.77 - ETA: 48s - loss: 0.5012 - acc: 0.77 - ETA: 48s - loss: 0.5011 - acc: 0.77 - ETA: 48s - loss: 0.5008 - acc: 0.77 - ETA: 48s - loss: 0.5004 - acc: 0.77 - ETA: 48s - loss: 0.5004 - acc: 0.77 - ETA: 48s - loss: 0.5001 - acc: 0.77 - ETA: 48s - loss: 0.4997 - acc: 0.77 - ETA: 48s - loss: 0.4996 - acc: 0.77 - ETA: 48s - loss: 0.4995 - acc: 0.77 - ETA: 48s - loss: 0.4998 - acc: 0.77 - ETA: 48s - loss: 0.5000 - acc: 0.77 - ETA: 48s - loss: 0.4998 - acc: 0.77 - ETA: 47s - loss: 0.4996 - acc: 0.77 - ETA: 47s - loss: 0.4996 - acc: 0.77 - ETA: 47s - loss: 0.4999 - acc: 0.77 - ETA: 47s - loss: 0.4996 - acc: 0.77 - ETA: 47s - loss: 0.4994 - acc: 0.77 - ETA: 47s - loss: 0.4990 - acc: 0.77 - ETA: 47s - loss: 0.4993 - acc: 0.77 - ETA: 

5036/6090 [=======================>......] - ETA: 34s - loss: 0.5023 - acc: 0.77 - ETA: 33s - loss: 0.5022 - acc: 0.77 - ETA: 33s - loss: 0.5025 - acc: 0.77 - ETA: 33s - loss: 0.5024 - acc: 0.77 - ETA: 33s - loss: 0.5027 - acc: 0.77 - ETA: 33s - loss: 0.5024 - acc: 0.77 - ETA: 33s - loss: 0.5023 - acc: 0.77 - ETA: 33s - loss: 0.5027 - acc: 0.77 - ETA: 33s - loss: 0.5028 - acc: 0.77 - ETA: 33s - loss: 0.5025 - acc: 0.77 - ETA: 33s - loss: 0.5023 - acc: 0.77 - ETA: 33s - loss: 0.5024 - acc: 0.77 - ETA: 33s - loss: 0.5021 - acc: 0.77 - ETA: 32s - loss: 0.5018 - acc: 0.77 - ETA: 32s - loss: 0.5018 - acc: 0.77 - ETA: 32s - loss: 0.5020 - acc: 0.77 - ETA: 32s - loss: 0.5020 - acc: 0.77 - ETA: 32s - loss: 0.5019 - acc: 0.77 - ETA: 32s - loss: 0.5018 - acc: 0.77 - ETA: 32s - loss: 0.5016 - acc: 0.77 - ETA: 32s - loss: 0.5018 - acc: 0.77 - ETA: 32s - loss: 0.5018 - acc: 0.77 - ETA: 32s - loss: 0.5016 - acc: 0.77 - ETA: 32s - loss: 0.5015 - acc: 0.77 - ETA: 32s - loss: 0.5015 - acc: 0.77 - ETA: 

5896/6090 [============================>.] - ETA: 18s - loss: 0.5006 - acc: 0.77 - ETA: 18s - loss: 0.5008 - acc: 0.77 - ETA: 18s - loss: 0.5009 - acc: 0.77 - ETA: 18s - loss: 0.5007 - acc: 0.77 - ETA: 18s - loss: 0.5006 - acc: 0.77 - ETA: 18s - loss: 0.5004 - acc: 0.77 - ETA: 18s - loss: 0.5003 - acc: 0.77 - ETA: 18s - loss: 0.5003 - acc: 0.77 - ETA: 18s - loss: 0.5003 - acc: 0.77 - ETA: 18s - loss: 0.5001 - acc: 0.77 - ETA: 17s - loss: 0.4999 - acc: 0.77 - ETA: 17s - loss: 0.5001 - acc: 0.77 - ETA: 17s - loss: 0.4998 - acc: 0.77 - ETA: 17s - loss: 0.5000 - acc: 0.77 - ETA: 17s - loss: 0.4998 - acc: 0.77 - ETA: 17s - loss: 0.4996 - acc: 0.77 - ETA: 17s - loss: 0.4996 - acc: 0.77 - ETA: 17s - loss: 0.5000 - acc: 0.77 - ETA: 17s - loss: 0.5000 - acc: 0.77 - ETA: 17s - loss: 0.5004 - acc: 0.77 - ETA: 17s - loss: 0.5002 - acc: 0.77 - ETA: 17s - loss: 0.5002 - acc: 0.77 - ETA: 17s - loss: 0.5000 - acc: 0.77 - ETA: 17s - loss: 0.4998 - acc: 0.77 - ETA: 16s - loss: 0.5001 - acc: 0.77 - ETA: 

6090/6090 [==============================] - ETA: 3s - loss: 0.4982 - acc: 0.777 - ETA: 3s - loss: 0.4984 - acc: 0.777 - ETA: 3s - loss: 0.4982 - acc: 0.777 - ETA: 3s - loss: 0.4981 - acc: 0.777 - ETA: 3s - loss: 0.4979 - acc: 0.778 - ETA: 3s - loss: 0.4979 - acc: 0.778 - ETA: 2s - loss: 0.4977 - acc: 0.778 - ETA: 2s - loss: 0.4980 - acc: 0.778 - ETA: 2s - loss: 0.4980 - acc: 0.778 - ETA: 2s - loss: 0.4982 - acc: 0.778 - ETA: 2s - loss: 0.4979 - acc: 0.778 - ETA: 2s - loss: 0.4977 - acc: 0.778 - ETA: 2s - loss: 0.4982 - acc: 0.778 - ETA: 2s - loss: 0.4982 - acc: 0.778 - ETA: 2s - loss: 0.4983 - acc: 0.777 - ETA: 2s - loss: 0.4983 - acc: 0.777 - ETA: 2s - loss: 0.4982 - acc: 0.778 - ETA: 2s - loss: 0.4980 - acc: 0.778 - ETA: 2s - loss: 0.4980 - acc: 0.778 - ETA: 2s - loss: 0.4982 - acc: 0.778 - ETA: 1s - loss: 0.4980 - acc: 0.778 - ETA: 1s - loss: 0.4982 - acc: 0.778 - ETA: 1s - loss: 0.4984 - acc: 0.778 - ETA: 1s - loss: 0.4986 - acc: 0.777 - ETA: 1s - loss: 0.4984 - acc: 0.778 - ETA: 

In [79]:
pred=model.predict(X_test)

In [80]:
f1_score(y_test.as_matrix(),np.round(pred).astype(int))

C:\Users\gohw\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


0.715092816787732

# Bigram Word2Vec

In [123]:
bigram = Phrases(corpus, min_count=1, threshold=2)

In [124]:
model = Word2Vec(Phraser(bigram)[corpus], iter=100, min_count=1, size=300, window=5)

In [125]:
len(model.wv.index2word)

18588

In [126]:
model.wv.syn0.shape

C:\Users\gohw\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


(18588, 300)

In [140]:
seq = [[model.wv.index2word.index(word)+1 for word in line] for line in Phraser(bigram)[corpus]]

In [144]:
pad = pad_sequences(seq, maxlen=30, truncating='post', padding='post')

In [145]:
pad.shape

(7613, 30)

In [146]:
emb_mat_bi = np.zeros((len(model.wv.index2word)+1,300))
emb_mat_bi[1:] = model.wv.syn0

C:\Users\gohw\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  


In [147]:
print(emb_mat_bi.shape)
emb_mat_bi

(18589, 300)


array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.41050050e-01,  1.02071381e+00, -1.72832698e-01, ...,
        -8.06990087e-01,  4.93431777e-01, -8.95636141e-01],
       [-1.79948181e-01,  7.80899107e-01, -1.29675508e+00, ...,
         2.13270998e+00, -1.14702332e+00,  5.83924115e-01],
       ...,
       [-1.03534691e-01,  5.11198901e-02,  1.20399781e-02, ...,
         1.34615935e-02,  3.03649921e-02, -9.97772589e-02],
       [ 4.40160185e-02,  1.33806720e-01, -1.36804566e-01, ...,
        -1.58327408e-02,  1.04513116e-01, -2.89176678e-04],
       [ 3.59053500e-02,  9.10307914e-02, -9.10026133e-02, ...,
        -2.49990653e-02,  7.95121416e-02, -2.54430022e-04]])

In [148]:
model = Sequential()
embedding = Embedding(emb_mat_bi.shape[0], emb_mat_bi.shape[1], embeddings_initializer=Constant(emb_mat_bi), input_length=30, trainable=False)
model.add(embedding)
#model.add(SpatialDropout1D(0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
#model.add(Flatten())
#model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
opt = Adam(lr=1e-5)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [149]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 30, 300)           5576700   
_________________________________________________________________
lstm_6 (LSTM)                (None, 64)                93440     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 65        
Total params: 5,670,205
Trainable params: 93,505
Non-trainable params: 5,576,700
_________________________________________________________________


In [150]:
X_train, X_test, y_train, y_test = train_test_split(pad,df['target'],test_size=0.2, shuffle=True)

In [151]:
history = model.fit(X_train, y_train, batch_size=4, epochs=5, validation_data=(X_test, y_test))

Train on 6090 samples, validate on 1523 samples
Epoch 1/5


 808/6090 [==>...........................] - ETA: 5:52:36 - loss: 0.6932 - acc: 0.50 - ETA: 2:58:22 - loss: 0.6924 - acc: 0.50 - ETA: 1:59:49 - loss: 0.6921 - acc: 0.58 - ETA: 1:30:41 - loss: 0.6929 - acc: 0.56 - ETA: 1:13:02 - loss: 0.6926 - acc: 0.55 - ETA: 1:01:24 - loss: 0.6924 - acc: 0.54 - ETA: 53:04 - loss: 0.6926 - acc: 0.5000 - ETA: 46:48 - loss: 0.6926 - acc: 0.50 - ETA: 41:56 - loss: 0.6927 - acc: 0.47 - ETA: 38:00 - loss: 0.6927 - acc: 0.47 - ETA: 34:49 - loss: 0.6927 - acc: 0.45 - ETA: 32:08 - loss: 0.6927 - acc: 0.45 - ETA: 29:54 - loss: 0.6928 - acc: 0.42 - ETA: 27:56 - loss: 0.6928 - acc: 0.41 - ETA: 26:17 - loss: 0.6927 - acc: 0.41 - ETA: 24:50 - loss: 0.6928 - acc: 0.42 - ETA: 23:32 - loss: 0.6928 - acc: 0.42 - ETA: 22:22 - loss: 0.6931 - acc: 0.40 - ETA: 21:22 - loss: 0.6929 - acc: 0.40 - ETA: 20:37 - loss: 0.6930 - acc: 0.38 - ETA: 19:46 - loss: 0.6930 - acc: 0.40 - ETA: 19:00 - loss: 0.6930 - acc: 0.38 - ETA: 18:18 - loss: 0.6931 - acc: 0.38 - ETA: 17:39 - loss: 0.

1624/6090 [=======>......................] - ETA: 4:33 - loss: 0.6930 - acc: 0.514 - ETA: 4:33 - loss: 0.6930 - acc: 0.514 - ETA: 4:32 - loss: 0.6930 - acc: 0.514 - ETA: 4:31 - loss: 0.6930 - acc: 0.514 - ETA: 4:31 - loss: 0.6930 - acc: 0.513 - ETA: 4:30 - loss: 0.6930 - acc: 0.515 - ETA: 4:29 - loss: 0.6930 - acc: 0.516 - ETA: 4:29 - loss: 0.6930 - acc: 0.515 - ETA: 4:28 - loss: 0.6930 - acc: 0.515 - ETA: 4:27 - loss: 0.6930 - acc: 0.514 - ETA: 4:26 - loss: 0.6930 - acc: 0.514 - ETA: 4:26 - loss: 0.6930 - acc: 0.515 - ETA: 4:25 - loss: 0.6930 - acc: 0.515 - ETA: 4:25 - loss: 0.6930 - acc: 0.513 - ETA: 4:24 - loss: 0.6930 - acc: 0.515 - ETA: 4:23 - loss: 0.6930 - acc: 0.514 - ETA: 4:23 - loss: 0.6930 - acc: 0.516 - ETA: 4:22 - loss: 0.6930 - acc: 0.515 - ETA: 4:21 - loss: 0.6930 - acc: 0.514 - ETA: 4:21 - loss: 0.6930 - acc: 0.515 - ETA: 4:20 - loss: 0.6930 - acc: 0.516 - ETA: 4:19 - loss: 0.6930 - acc: 0.517 - ETA: 4:19 - loss: 0.6930 - acc: 0.517 - ETA: 4:18 - loss: 0.6930 - acc: 0.5

2440/6090 [===========>..................] - ETA: 3:04 - loss: 0.6927 - acc: 0.541 - ETA: 3:03 - loss: 0.6927 - acc: 0.541 - ETA: 3:03 - loss: 0.6927 - acc: 0.540 - ETA: 3:03 - loss: 0.6927 - acc: 0.539 - ETA: 3:03 - loss: 0.6927 - acc: 0.539 - ETA: 3:02 - loss: 0.6927 - acc: 0.539 - ETA: 3:02 - loss: 0.6927 - acc: 0.540 - ETA: 3:02 - loss: 0.6927 - acc: 0.540 - ETA: 3:01 - loss: 0.6927 - acc: 0.539 - ETA: 3:01 - loss: 0.6927 - acc: 0.540 - ETA: 3:01 - loss: 0.6927 - acc: 0.540 - ETA: 3:01 - loss: 0.6927 - acc: 0.540 - ETA: 3:00 - loss: 0.6927 - acc: 0.541 - ETA: 3:00 - loss: 0.6927 - acc: 0.541 - ETA: 3:00 - loss: 0.6927 - acc: 0.541 - ETA: 3:00 - loss: 0.6927 - acc: 0.542 - ETA: 2:59 - loss: 0.6927 - acc: 0.542 - ETA: 2:59 - loss: 0.6927 - acc: 0.542 - ETA: 2:59 - loss: 0.6927 - acc: 0.542 - ETA: 2:58 - loss: 0.6926 - acc: 0.543 - ETA: 2:58 - loss: 0.6926 - acc: 0.542 - ETA: 2:58 - loss: 0.6926 - acc: 0.542 - ETA: 2:58 - loss: 0.6926 - acc: 0.543 - ETA: 2:57 - loss: 0.6926 - acc: 0.5

3256/6090 [===============>..............] - ETA: 2:17 - loss: 0.6923 - acc: 0.554 - ETA: 2:17 - loss: 0.6923 - acc: 0.553 - ETA: 2:16 - loss: 0.6923 - acc: 0.553 - ETA: 2:16 - loss: 0.6923 - acc: 0.553 - ETA: 2:16 - loss: 0.6923 - acc: 0.553 - ETA: 2:16 - loss: 0.6923 - acc: 0.553 - ETA: 2:16 - loss: 0.6923 - acc: 0.553 - ETA: 2:15 - loss: 0.6922 - acc: 0.554 - ETA: 2:15 - loss: 0.6922 - acc: 0.554 - ETA: 2:15 - loss: 0.6922 - acc: 0.554 - ETA: 2:15 - loss: 0.6922 - acc: 0.553 - ETA: 2:15 - loss: 0.6922 - acc: 0.553 - ETA: 2:14 - loss: 0.6922 - acc: 0.553 - ETA: 2:14 - loss: 0.6922 - acc: 0.554 - ETA: 2:14 - loss: 0.6923 - acc: 0.553 - ETA: 2:14 - loss: 0.6923 - acc: 0.553 - ETA: 2:14 - loss: 0.6923 - acc: 0.553 - ETA: 2:13 - loss: 0.6923 - acc: 0.552 - ETA: 2:13 - loss: 0.6923 - acc: 0.552 - ETA: 2:13 - loss: 0.6922 - acc: 0.553 - ETA: 2:13 - loss: 0.6922 - acc: 0.553 - ETA: 2:13 - loss: 0.6922 - acc: 0.553 - ETA: 2:13 - loss: 0.6923 - acc: 0.552 - ETA: 2:12 - loss: 0.6922 - acc: 0.5

...

3768/6090 [=================>............] - ETA: 1:19 - loss: 0.6178 - acc: 0.685 - ETA: 1:19 - loss: 0.6182 - acc: 0.685 - ETA: 1:19 - loss: 0.6180 - acc: 0.685 - ETA: 1:19 - loss: 0.6179 - acc: 0.685 - ETA: 1:19 - loss: 0.6179 - acc: 0.685 - ETA: 1:19 - loss: 0.6177 - acc: 0.6855

C:\Users\gohw\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.112750). Check your callbacks.
  % delta_t_median)


4596/6090 [=====================>........] - ETA: 1:19 - loss: 0.6176 - acc: 0.685 - ETA: 1:18 - loss: 0.6178 - acc: 0.685 - ETA: 1:18 - loss: 0.6177 - acc: 0.685 - ETA: 1:18 - loss: 0.6176 - acc: 0.685 - ETA: 1:18 - loss: 0.6179 - acc: 0.685 - ETA: 1:18 - loss: 0.6177 - acc: 0.685 - ETA: 1:18 - loss: 0.6177 - acc: 0.685 - ETA: 1:18 - loss: 0.6175 - acc: 0.685 - ETA: 1:18 - loss: 0.6175 - acc: 0.685 - ETA: 1:17 - loss: 0.6177 - acc: 0.685 - ETA: 1:17 - loss: 0.6175 - acc: 0.685 - ETA: 1:17 - loss: 0.6174 - acc: 0.685 - ETA: 1:17 - loss: 0.6173 - acc: 0.685 - ETA: 1:17 - loss: 0.6172 - acc: 0.685 - ETA: 1:17 - loss: 0.6171 - acc: 0.686 - ETA: 1:17 - loss: 0.6170 - acc: 0.686 - ETA: 1:16 - loss: 0.6170 - acc: 0.686 - ETA: 1:16 - loss: 0.6171 - acc: 0.686 - ETA: 1:16 - loss: 0.6167 - acc: 0.686 - ETA: 1:16 - loss: 0.6168 - acc: 0.686 - ETA: 1:16 - loss: 0.6165 - acc: 0.686 - ETA: 1:16 - loss: 0.6164 - acc: 0.686 - ETA: 1:16 - loss: 0.6168 - acc: 0.686 - ETA: 1:16 - loss: 0.6168 - acc: 0.6

5456/6090 [=========================>....] - ETA: 50s - loss: 0.6172 - acc: 0.68 - ETA: 50s - loss: 0.6172 - acc: 0.68 - ETA: 50s - loss: 0.6171 - acc: 0.68 - ETA: 50s - loss: 0.6172 - acc: 0.68 - ETA: 50s - loss: 0.6174 - acc: 0.68 - ETA: 50s - loss: 0.6174 - acc: 0.68 - ETA: 50s - loss: 0.6174 - acc: 0.68 - ETA: 49s - loss: 0.6172 - acc: 0.68 - ETA: 49s - loss: 0.6173 - acc: 0.68 - ETA: 49s - loss: 0.6172 - acc: 0.68 - ETA: 49s - loss: 0.6172 - acc: 0.68 - ETA: 49s - loss: 0.6172 - acc: 0.68 - ETA: 49s - loss: 0.6173 - acc: 0.68 - ETA: 49s - loss: 0.6172 - acc: 0.68 - ETA: 48s - loss: 0.6173 - acc: 0.68 - ETA: 48s - loss: 0.6173 - acc: 0.68 - ETA: 48s - loss: 0.6174 - acc: 0.68 - ETA: 48s - loss: 0.6173 - acc: 0.68 - ETA: 48s - loss: 0.6173 - acc: 0.68 - ETA: 48s - loss: 0.6174 - acc: 0.68 - ETA: 48s - loss: 0.6172 - acc: 0.68 - ETA: 47s - loss: 0.6173 - acc: 0.68 - ETA: 47s - loss: 0.6172 - acc: 0.68 - ETA: 47s - loss: 0.6173 - acc: 0.68 - ETA: 47s - loss: 0.6172 - acc: 0.68 - ETA: 

6090/6090 [==============================] - ETA: 21s - loss: 0.6166 - acc: 0.68 - ETA: 21s - loss: 0.6166 - acc: 0.68 - ETA: 21s - loss: 0.6165 - acc: 0.68 - ETA: 21s - loss: 0.6167 - acc: 0.68 - ETA: 20s - loss: 0.6167 - acc: 0.68 - ETA: 20s - loss: 0.6166 - acc: 0.68 - ETA: 20s - loss: 0.6164 - acc: 0.68 - ETA: 20s - loss: 0.6163 - acc: 0.68 - ETA: 20s - loss: 0.6167 - acc: 0.68 - ETA: 20s - loss: 0.6165 - acc: 0.68 - ETA: 20s - loss: 0.6164 - acc: 0.68 - ETA: 19s - loss: 0.6165 - acc: 0.68 - ETA: 19s - loss: 0.6164 - acc: 0.68 - ETA: 19s - loss: 0.6164 - acc: 0.68 - ETA: 19s - loss: 0.6163 - acc: 0.68 - ETA: 19s - loss: 0.6163 - acc: 0.68 - ETA: 19s - loss: 0.6163 - acc: 0.68 - ETA: 19s - loss: 0.6167 - acc: 0.68 - ETA: 19s - loss: 0.6165 - acc: 0.68 - ETA: 18s - loss: 0.6165 - acc: 0.68 - ETA: 18s - loss: 0.6163 - acc: 0.68 - ETA: 18s - loss: 0.6163 - acc: 0.68 - ETA: 18s - loss: 0.6162 - acc: 0.68 - ETA: 18s - loss: 0.6161 - acc: 0.68 - ETA: 18s - loss: 0.6164 - acc: 0.68 - ETA: 

In [152]:
pred=model.predict(X_test)

In [153]:
f1_score(y_test.as_matrix(),np.round(pred).astype(int))

C:\Users\gohw\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


0.673544583640383